Automatic Differentiation Package - Torch AUTOGRAD
============================================

### torch.autograd.no_grad

In [2]:
import torch

In [5]:
x = torch.tensor([1.0], requires_grad=True)
with torch.no_grad():
    y = x*2
y.requires_grad

False

In [6]:
@torch.no_grad()
def doubler(x):
    return x*2
z = doubler(x)
z.requires_grad

False

### torch.autograd.enable_grad

In [8]:
x = torch.tensor([1.0], requires_grad=True)
with torch.no_grad():
    with torch.enable_grad():
        y = x*2
print(y.requires_grad)
y.backward()
print(x.grad)

True
tensor([2.])


In [9]:
@torch.enable_grad()
def doubler(x):
    return x*2
z = doubler(x)
z.requires_grad

True

### torch.autograd.set_grad_enabled(mode)

In [10]:
x = torch.tensor([1.0], requires_grad=True)
is_train = False
with torch.set_grad_enabled(is_train):
    y = x*2
print(y.requires_grad)

torch.set_grad_enabled(True)
y = x * 2
print(y.requires_grad)

torch.set_grad_enabled(False)
y = x * 2
print(y.requires_grad)

False
True
False


## tensor autograd function

In [11]:
# is_leaf
a = torch.rand(10, requires_grad=True)
print(a.is_leaf)

b = torch.rand(10, requires_grad=True).cuda()
print(b.is_leaf)
# b was created by the operation that cast a cpu Tnesor into a cuda Tensor

c = torch.rand(10, requires_grad=True) + 2
print(c.is_leaf)
# c was created by the addition operation

d = torch.rand(10).cuda()
print(d.is_leaf)
# d does not rquire gradients and so has no operation creating it

e = torch.rand(10).cuda().requires_grad_()
print(e.is_leaf)

f = torch.rand(10, requires_grad=True, device="cuda")
print(f.is_leaf)

True
True
True
True
True
True


In [13]:
# register_hook(hook)
v = torch.tensor([0., 0., 0.], requires_grad=True)
h = v.register_hook(lambda grad: grad * 2)    # double the gradient
v.backward(torch.tensor([1., 2., 3.]))
print(v.grad)
h.remove()    # removes the hook
print(v.grad)

tensor([2., 4., 6.])
tensor([2., 4., 6.])


## profiler

In [21]:
x = torch.randn( (1, 1), requires_grad=True)
#with torch.autograd.profiler.profile() as prof:
#    y = x ** 2;
#    y.backward()    # runtime error
#print(prof)

IndentationError: unexpected indent (<ipython-input-21-348aedd72811>, line 4)

## anomaly detectoin

In [22]:
import torch
from torch import autograd
class MyFunc(autograd.Function):
    @staticmethod
    def forward(ctx, inp):
        return inp.clone()
    @staticmethod
    def backward(ctx, gO):
        # Error during the backward pass
        raise RuntimeError("Some error in backward")
        return gO.clone()
    
def run_fn(a):
    out = MyFunc.apply(a)
    return out.sum()

inp = torch.rand(10, 10, requires_grad=True)
out = run_fn(inp)
out.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [23]:
with autograd.detect_anomaly():
    inp = torch.rand(10, 10, requires_grad=True)
    out = run_fn(inp)
    out.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn